In [1]:
import numpy as np
import pandas as pd
from faker import Faker
import matplotlib.pyplot as plt
from IPython.display import clear_output
import random

# Create a list of student names
fake = Faker() # use faker in order to generate random students' names
Students=[]
x=0
while x<100:
    name=fake.first_name()
    if name not in Students:
        Students.append(name)
        x+=1
    else:
        continue
        

# Dictationary to store each subject's mean and standard deviation
subject_params = {
    "Math": (75, 12),      
    "Art": (70, 10),       
    "Physics": (70, 15),    
    "Sport": (80, 8)        
}

# Create inital dataFrame- grades of 100 students in all subjects
data = []
for student in Students:
    for subject, params in subject_params.items():
        # Get mean and std from subject_params
        mean, std_dev = params
        
        # Generate a random Grade between 0 and 100 and make it distributed normally 
        Grade = int(np.random.normal(mean, std_dev))        
        Grade = max(0, min(Grade, 100))
        
        # Add student's data to list
        data.append([student, subject, Grade])

df = pd.DataFrame(data, columns=["Student", "Subject", "Grade"])

#Create 3 main variables that will be used in various functions
Student=pd.Series(df['Student'].unique(),name='Student')
Subject=pd.Series(df['Subject'].unique(),name='Subject')
#dictationary to store variables that will be declared in one function and be used in other (and not be given as a parameter)
global_vars={'student_name':None,'subject_name':None,'Grade':None}

In [2]:
#Main menu
def intro():
    print("Welcome!")
    print("I'm here to help teachers manage their student's grades in 4 Subjects: Math,Physics,Sport and Art.")
    print("I already have grades of 100 students, you can add new grades,students and subjects.")
    print("Also I can provide the average grade of each student and subject and provide statistics on the student's performance")
    print("in each subject, I can even calculate the top 5 students in all subjects.")
    print("Let's start!")
    main_menu()

def main_menu():
    while True:
        print("\nWhat do you want me to do:")
        print("1) Show me the average grade in each subject")
        print("2) Show me the avergae grade of specific student")
        print("3) Add new grade")
        print("4) Add new subject")
        print("5) Add new student")
        print("6) Correct grade")
        print("7) Provide statistics on specific subject")
        print("8) Show me the top 5 students")
        print("9) Exit")
        next_action=input("Enter the number of the required action: ")
        try:
            next_action = int(next_action)
        except ValueError:
            print("Invalid input. Please enter a number.")
            continue
            
        match next_action:
            case 1:
                clear_output(wait=True)
                avg_per_subject()
            case 2:
                clear_output(wait=True)
                avg_student()
            case 3:
                clear_output(wait=True)
                add_grade()
            case 4:
                clear_output(wait=True)
                add_new_subject()
            case 5:
                clear_output(wait=True)
                add_new_student()
            case 6:
                clear_output(wait=True)
                correct_grade()
            case 7:
                clear_output(wait=True)
                statistics_by_subject()
            case 8:
                clear_output(wait=True)
                top_5_students()
            case 9:
                print("Bye Bye!")
                return
            case _:
                print("Invalid option. Please select a valid action (1-9).")

In [3]:
#Provide the avg score for each subject
def avg_per_subject():
    while True:
        print("Here are the average score in each Subject: ")
        print(df.groupby('Subject')['Grade'].mean(), end='\n')
        main_menu()
        break

In [4]:
#The user enter the student's name and gets his/her avg grade
def avg_student():
    while True:
        student_name=input('Student name: (student for example:{})'.format(Student[random.randint(0,99)])).title().strip()
        if student_name in Student.values:
            print("{} has average grade of {} in {} tests".format(student_name,df.loc[df['Student']==student_name]['Grade'].mean(),len(df.loc[df['Student']==student_name])))
        else:
            print("There isn't student called {}".format(student_name,))
        next_action= input("Do you want to check another student's average? (yes/no): ").strip().lower()
        if next_action != 'yes':
            main_menu()
            break

In [5]:
def add_grade(student_name=None,subject_name=None):
    global df,global_vars
    while True:
        if student_name is None:
            student_name=input('Student name: (student for example:{}) '.format(Student[random.randint(0,99)])).title().strip()
            global_vars['student_name']=student_name
        if validation(student_name,Student):
            if subject_name is None:
                subject_name=input('Subject: (subject for example:{}) '.format(Subject[random.randint(0,3)])).title().strip()
                global_vars['subject_name']=subject_name
            if validation(subject_name,Subject):
                if grade_duplication_validation(student_name,subject_name):
                    grade=int(input("Grade: ").strip())
                    global_vars['grade']=grade
                    if grade_validation(grade):
                        new_grade=pd.Series({'Student':global_vars['student_name'],'Subject':global_vars['subject_name'],'Grade':global_vars['grade']})
                        df=pd.concat([df,new_grade.to_frame().T],ignore_index=True)
                        print("we updated that {} gets {} in {}".format(global_vars['student_name'],global_vars['grade'],global_vars['subject_name']))
        next_action= input("Do you want to add another grade? (yes/no): ").strip().lower()
        if next_action != 'yes':
            main_menu()
            break

#validate that the student/subject name exists and if not it will refer the user to create it
def validation(name,column):
    global global_vars
    if name in column.values:
        return True
    else:
        print("There isn't {} in the {}'s list".format(name,column.name))
        print("What do you want to do: ")
        print("1) Add {} as new {}".format(name,column.name))
        print("2) I want to correct the {}'s name".format(column.name))
        print("3) Return to the main menu")
        next_action=input("Enter the number of the desired action ")
        match next_action:
            case '1':
                if column.name=='Student':
                    add_new_student(name)
                    if validation(name,Student):
                        return True
                else:
                    add_new_subject(name)
                    if validation(name,Subject):
                        return True
            case '2':
                name=input("New {}'s name: ".format(column.name)).title().strip()
                if column.name=="Student":
                    global_vars['student_name']=name
                    add_grade(student_name=global_vars['student_name'])
                else:
                    global_vars['subject_name']=name
                    add_grade(student_name=global_vars['student_name'],subject_name=global_vars['subject_name'])
            case '3':
                main_menu()
                return

#validate the grade is valid
def grade_validation(grade):
    global global_vars
    if grade>=0 and grade<=100:
        print('grade validation passed')
        return True
    else:
        print("The student's grade has to be a number between 0 and 100")
        x=0
        while x<3:
            try:
                print("Re-enter the grade, you have {} attempts left:".format(3-x))
                x+=1
                grade=int(input("Grade:").strip())
                global_vars['grade']=grade
                if grade>=0 and grade<=100:
                    print('true')
                    return True
                else:
                    print("The student's grade has to be a number between 0 and 100")
            except ValueError:
                print("The student's grade has to be a number between 0 and 100")
        print("Check again the student's grade and try again")
        return False 

#will be used in case the user wants to add grade that is already exist (same subject and student name)
def grade_duplication_validation(student_name,subject_name):
    global df
    existing_row=df[(df['Student']==student_name)&(df['Subject']==subject_name)]
    if existing_row.shape[0]>0:# shape[0] returns the # of rows for the specific student&subject, >0 means that a grade is already exists
        print("{} already has grade of {} in {}".format(student_name,int(existing_row['Grade']),subject_name))
        print("Do you want to correct the grade?")
        if_correct=input("Write 'yes' to correct or anything else to return to main menu").title().strip()
        if if_correct=='Yes':
            df=df[~((df['Student']==student_name)&(df['Subject']==subject_name))]
            return True
        else:
            return False
    else:
        return True

In [6]:
def add_new_subject(name=None):
    global Subject
    while True:
        if name is None:
            name=input("{}'s Name:".format(Subject.name)).title().strip()

        if name in Subject.values:
            print("There is already {} in the {}'s list".format(name,Subject.name))
            break
        else:
            try:
                if name.isalpha():#the only validation for subject name is that it only contains letters 
                    new_value=pd.Series(name,name=Subject.name)
                    Subject=pd.concat([Subject,new_value],ignore_index=True)
                    print("{} is added to the {}'s list".format(name,Subject.name))
                else:
                    raise ValueError("{}'s Name can only contain letters".format(Subject.name))
            except ValueError as e:
                print(e)
        next_action= input("Do you want to add another subject? (yes/no): ").strip().lower()
        if next_action != 'yes':
            main_menu()
            break

In [7]:
def add_new_student(name=None):
    global Student
    while True:
        if name is None:
            name=input("{}'s Name:".format(Student.name)).title().strip()

        if name in Student.values:
            print("There is already {} in the {}'s list".format(name,Student.name))
            main_menu()
        else:
            try:
                if name.isalpha():#the only validation for student name is that it only contains letters 
                    new_value=pd.Series(name,name=Student.name)
                    Student=pd.concat([Student,new_value],ignore_index=True)
                    print("{} is added to the {}'s list".format(name,Student.name))
                    main_menu()
                else:
                    raise ValueError("{}'s Name can only contain letters".format(Student.name))
                    add_new_student()
            except ValueError as e:
                print(e)
        next_action= input("Do you want to add another student? (yes/no): ").strip().lower()
        if next_action != 'yes':
            main_menu()
            break

In [8]:
def correct_grade():
    global df,global_vars
    while True:
        student_name=input('Student name: (student for example:{})'.format(Student[random.randint(0,99)])).title().strip()
        
        #validate that the name exists in the student's list
        if validation(student_name,Student):
            global_vars['student_name']=student_name
            subject_name=input('Subject: (subject for example:{})'.format(Subject[random.randint(0,3)])).title().strip()
            
            #validate that the name exists in the subject's list
            if validation(subject_name,Subject):
                global_vars['subject_name']=subject_name 
                current_grade=df[(df['Student']==student_name)&(df['Subject']==subject_name)]
                
                #check if the student already has grade in this subject
                if current_grade.shape[0]>0:
                    print("{} has grade of {} in {}, what is the updated grade?".format(student_name,int(current_grade['Grade']),subject_name))
                    grade=int(input("Enter new grade ").strip())
                    global_vars['grade']=grade
                    if grade_validation(grade):
                        
                        #update the new grade
                        df.loc[(df['Student'] == global_vars['student_name']) & (df['Subject'] == global_vars['subject_name']), 'Grade'] = global_vars['grade']
                        print("we updated that {} gets {} in {}".format(global_vars['student_name'],global_vars['grade'],global_vars['subject_name']))
                    next_action= input("Do you want to correct another student's grade? (yes/no): ").strip().lower()
                    if next_action != 'yes':
                        main_menu()
                        break
                else:
                    next_action=input("{} doesn't have a grade in {}, do you want to enter new grade?".format(global_vars['student_name'],global_vars['subject_name'])).strip().lower()
                    if next_action != 'yes':
                        main_menu()
                        break
                    else:
                        add_grade(student_name=global_vars['student_name'],subject_name=global_vars['subject_name'])
                        break

In [9]:
#provide per subject: grade's histogram,grade distribution by groups and statistics using describe's function
def statistics_by_subject():
    global global_vars
    while True:
        print('For which subject you want to get info on:\n{}'.format('\n'.join(Subject.values)))    
        subject_name=input('subject: (subject for example:{})'.format(Subject[random.randint(0,3)])).title().strip()
        if subject_name not in Subject.values:
            print("There isn't subject called {}, return to main menu and add {} as subject and add the students' grades".format(subject_name,subject_name))
            main_menu()
            break
        else:
            global_vars['subject_name']=subject_name
            grade_historgram(global_vars['subject_name'])
            grade_groups(global_vars['subject_name'])
            general_info(global_vars['subject_name'])
        break
        
def grade_historgram(subject_name):
    df[df['Subject']==subject_name]['Grade'].plot(kind='hist',title="{}'s' grade distribution".format(subject_name))
    plt.show()

def grade_groups(subject_name):
    bins=[0,50,70,80,90,101]
    labels=['<50','51-70','71-80','81-90','91-100']
    
    grades = df[df['Subject'] == subject_name]
    result = grades.groupby(pd.cut(grades['Grade'], bins=bins, labels=labels)).size()
    print('Grades distribution by groups:')
    for index, row in result.to_frame().iterrows():
        print("{}: {}".format(index,row.values[0]))

def general_info(subject_name):
    info=round(df[df['Subject']==subject_name].describe(),2)
    info.index=['Number of tests:','Average:','Std:','Lowest grade:','25% grade','50% grade','75% grade','Highest grade']
    print(info)

In [10]:
#display top 5 students by z-score's calculation (each subject gets equal weight) and their class ranks in each subject
def top_5_students():
    while True:
        avg=df.groupby('Subject')['Grade'].mean()
        std=df.groupby('Subject')['Grade'].std()
        data=[]
        for student in Student.values:
            for subject in Subject.values:
                
                #calculates z score for each student's grade
                Zscore=z_score(student,subject,avg,std)
                
                #calculate class rank in each subject
                class_rank=rank_in_class(student,subject)
                
                #append data in a list
                data.append([student,subject,Zscore,class_rank])
        
        #create new df with each student z-score and class rank in each subject
        df_z_score=pd.DataFrame(data=data,columns=['Student','Subject','Z-score','class_rank'])
        
        #this function returns df with the total z-score for each student in all subject
        df_z_score=total_z_score(df_z_score)
        
        print("Here are the top 5 students based on Z-score calculation and their class' rank in each subject")
        print("(For example: position 8 in Art means that this student got the 8th highest grade in Art)\n")
        print(df_z_score.head())
        break
    

#student's z-score in each subject
def z_score(student,subject,avg,std):
    student_score=int(df[(df['Student']==student)&(df['Subject']==subject)]['Grade'])
    #z-score= (grade-avg)/std
    Zscore=round(float(student_score-avg[subject])/std[subject],2)
    return Zscore

#student's class rank in each subject
def rank_in_class(student,subject):
    subject_grades=sorted(df[df['Subject']==subject]['Grade'].to_list(),reverse=True)
    rank=subject_grades.index(df[(df['Student']==student)&(df['Subject']==subject)]['Grade'].values[0])+1
    return rank

#sum all student z-score in order to display the top students in class
def total_z_score(df_z_score):
    #create new df with the each student's total z-score 
    total_z_score=df_z_score.groupby('Student')['Z-score'].sum().reset_index()
    total_z_score.columns=['Student','Total Z-score']
    
    #merge new df to the df in the parameter, use merge to keep indexes
    df_z_score=df_z_score.merge(total_z_score,on='Student')
    
    #use pivot to get each subject's class rank in seperate column
    df_z_score=df_z_score.pivot(columns='Subject',index=['Student','Total Z-score'],values='class_rank').sort_values(by='Total Z-score',ascending=False).reset_index()
    return df_z_score

In [12]:
intro()

Here are the top 5 students based on Z-score calculation and their class' rank in each subject
(For example: position 8 in Art means that this student got the 8th highest grade in Art)

Subject   Student  Total Z-score  Art  Math  Physics  Sport
0          Hannah           4.64    3     8       19     19
1            Adam           4.58   34    13        1     28
2           Nancy           4.20   28    19        2     38
3        Brittany           3.98   21    21       10     16
4          Carlos           3.75   45    37        9      2

What do you want me to do:
1) Show me the average grade in each subject
2) Show me the avergae grade of specific student
3) Add new grade
4) Add new subject
5) Add new student
6) Correct grade
7) Provide statistics on specific subject
8) Show me the top 5 students
9) Exit
Enter the number of the required action: 9
Bye Bye!
